In [6]:
from lifestore_file import lifestore_products, lifestore_sales, lifestore_searches
from login import loginn, validacion
import pandas as pd 
import altair as alt
import operator
import copy
from collections import OrderedDict
import operator

lifestore_searches = [id_search, id product]

lifestore_sales = [id_sale, id_product, score (from 1 to 5), date, refund (1 for true or 0 to false)]

lifestore_products = [id_product, name, price, category, stock]

In [80]:
stok_category = [ value[3:] for value in lifestore_products]
# print(stok_category)
diccionario_stok = {}
for value in  stok_category:
    if (value[0] not in diccionario_stok.keys()):
        diccionario_stok[value[0]]=value[1]
    diccionario_stok[value[0]]+= value[1]
    # app
print(diccionario_stok)


diccionario_stok = pd.DataFrame([[key, diccionario_stok[key]] for key in diccionario_stok.keys()], columns = ['category','stok'])
print(diccionario_stok)

domain = ['memorias usb', 'audifonos','pantallas','tarjetas de video','bocinas']
range_ = ['#d62728', '#1f77b4', '#17becf','#2ca02c', '#ff7f0e']

# Graficacion de la lista de productos
bars = alt.Chart(diccionario_stok).mark_bar().encode(
    x=alt.X('category', sort='y'),
    y="stok:Q",
    # color='category'
    color=alt.Color("category", legend=alt.Legend(title="Score"),scale=alt.Scale(domain=domain, range=range_)),
    
)

text = bars.mark_text(
    align='left',
    dx=-30,  # Nudges text to right so it doesn't appear on top of the bar
    color='black'
).encode(
    text='stok',
).properties(
            title='Stok de productos por categoria'
        )
(bars + text ).properties(height=900)

{'procesadores': 1837, 'tarjetas de video': 362, 'tarjetas madre': 824, 'discos duros': 441, 'memorias usb': 25, 'pantallas': 1196, 'bocinas': 147, 'audifonos': 656}
            category  stok
0       procesadores  1837
1  tarjetas de video   362
2     tarjetas madre   824
3       discos duros   441
4       memorias usb    25
5          pantallas  1196
6            bocinas   147
7          audifonos   656


alt.LayerChart(...)

In [2]:
sales_no_refound = [value for value in lifestore_sales if value[4] != 1 ]

print(type(sales_no_refound))
# print(sales_no_refound)
pd.DataFrame(sales_no_refound)

<class 'list'>


,0,1,2,3,4
0,1,1,5,24/07/2020,0
1,2,1,5,27/07/2020,0
2,3,2,5,24/02/2020,0
3,4,2,5,22/05/2020,0
4,5,2,5,01/01/2020,0
...,...,...,...,...,...
269,279,84,5,05/05/2020,0
270,280,85,5,05/05/2020,0
271,281,85,5,28/04/2020,0
272,282,89,3,06/01/2020,0


In [98]:
dataframe_products = pd.DataFrame(lifestore_products,columns=['id_product', 'name', 'price', 'category', 'stock'])
dataframe_searches = pd.DataFrame(lifestore_searches,columns=['id_search', 'id_product'])
dataframe_sales = pd.DataFrame(lifestore_sales,columns=['id_sale', 'id_product', 'score', 'date', 'refund' ])
sales_no_refound =pd.DataFrame(lifestore_sales,columns=['id_sale', 'id_product', 'score', 'date', 'refund' ])


In [120]:
print(dataframe_products.shape)
print(dataframe_searches.shape)
print(dataframe_sales.shape)

(96, 5)
(1033, 2)
(283, 5)


Categorías con menores ventas y categorías con menores búsquedas

In [92]:
join_sales = pd.merge(sales_no_refound,dataframe_products,on='id_product',how='left')#.groupby(['category']).head()
# print(join_sales)
join_sales = join_sales.groupby(['category']).count()['id_sale']
# dataframe_sales
print(join_sales)


category
audifonos              5
bocinas                2
discos duros          94
memorias usb           1
pantallas              2
procesadores         104
tarjetas de video     26
tarjetas madre        49
Name: id_sale, dtype: int64


In [122]:

join_searches = pd.merge(dataframe_searches,dataframe_products,on='id_product',how='left')#.groupby(['category']).head()
join_searches = join_searches.groupby(['category']).count()['name']
print(join_searches)

category
audifonos             64
bocinas                9
discos duros         463
pantallas             56
procesadores         222
tarjetas de video     82
tarjetas madre       137
Name: name, dtype: int64


In [172]:
# print(join_searches)
# print(join_sales)

test_vistas_compras=join_searches.div(join_sales).to_frame(name="vistas_x_compras").reset_index()
print(test_vistas_compras)
bar = alt.Chart(test_vistas_compras).mark_bar().encode(
    x='category',
    y='vistas_x_compras',
    color='category'
).properties(
            title='Promedio de vistas por compra'
        )


# test_vistas_compras = test_vistas_compras.rename(columns={'0':'vistas_x_compras',})
# # test_vistas_compras.columns
# print(test_vistas_compras.columns.values)

# Graficacion de la lista de productos
bars = alt.Chart(test_vistas_compras).mark_bar().encode(
    # x=alt.X('category', sort='y'),
    # y="vistas_x_compras:Q",
    x='category',
    y='vistas_x_compras:Q',
    color=alt.Color("category"),
    
)
text = bars.mark_text(
    align='left',
    dx=-30, 
    color='black'
).properties(
            title='Promedio de vistas por compra'
        )
(bars + text )



            category  vistas_x_compras
0          audifonos         12.800000
1            bocinas          4.500000
2       discos duros          4.925532
3       memorias usb               NaN
4          pantallas         28.000000
5       procesadores          2.134615
6  tarjetas de video          3.153846
7     tarjetas madre          2.795918


alt.LayerChart(...)

In [218]:
# dataframe_products
test_vistas_compras = test_vistas_compras.dropna()
testx = dataframe_products.groupby(['category']).sum()['stock']
# print(testx)
# print(test_vistas_compras)
ventas_x_vistas_años= testx * join_searches.div(join_sales)
# ventas_x_vistas_años = dataframe_products *test_vistas_compras
print(ventas_x_vistas_años)

category
audifonos             7334.400000
bocinas                657.000000
discos duros          2132.755319
memorias usb                  NaN
pantallas            33320.000000
procesadores          3887.134615
tarjetas de video     1100.692308
tarjetas madre        2275.877551
dtype: float64
